In [ ]:
# seq2seq를 이용해 글자 단위 번역기(영어->한국어)
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

batch_size = 64
epochs = 1000
latent_dim = 1024  # 인코딩 공간의 잠재 차원
data_path = 'kor.txt'
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
lines = open(data_path, encoding='utf8').read().split('\n')
# print(len(lines))

In [ ]:
#for line in lines[:len(lines) - 1]:
for line in lines[:2000]:
  input_text, target_text, _ = line.split('\t')
  target_text = '\t' + target_text + '\n'  # tab을 문장의 시작<Start Of Sentence>, enter를 문장의 끝<End Of Sentence>
  input_texts.append(input_text)
  target_texts.append(target_text)  # 영어
  for char in input_text:   # 영어
    if char not in input_characters:
      input_characters.add(char)   # 중복을 배제한 영어 글자 기억

  for char in target_text:   # 영어
    if char not in target_characters:
      target_characters.add(char)   # 중복을 배제한 한국어 글자 기억

print('input_texts : ', input_texts)
print('target_texts : ', target_texts)
print('input_characters : ',input_characters)
print('target_characters : ',target_characters)

# 리스트로 바꾸기
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))

num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)

print('특수문자 포함 영어 관련 글자 수 : ', num_encoder_tokens)
print('특수문자 포함 영어 관련 글자 : ', input_characters)

print('특수문자 포함 한국어 관련 글자 수 : ', num_decoder_tokens)
print('특수문자 포함 한국어 관련 글자 : ', target_characters)

max_encoder_seq_len = max([len(t) for t in input_texts])
max_decoder_seq_len = max([len(t) for t in target_texts])
print('영어 단어 중 가장 긴 단어 글자 수 : ', max_encoder_seq_len)
print('한국어 단어 중 가장 긴 단어 글자 수 : ', max_decoder_seq_len)

# 글자 집합에 글자 단위로 저장
input_token_index = dict([(char, i) for i, char in enumerate(input_characters) ])
print(input_token_index)  # 영어 :  {' ': 0, '!': 1, '"': 2, '$': 3, '%': 4, "'": 5, ',': 6, '-': 7,

target_token_index = dict([(char, i) for i, char in enumerate(target_characters) ])
print(target_token_index) # 한국어 : {'\t': 0, '\n': 1, ' ': 2, '!': 3, '"': 4, '%': 5, '(': 6, ')': 7,

encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_len, num_encoder_tokens), dtype='float32')
# print('encoder_input_data : ', encoder_input_data) # [[[0. 0. 0. ... 0. 0. 0.]  [0. 0. 0. ... 0. 0. 0.]
print(encoder_input_data.shape)   # (5892, 537, 74)

decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_len, num_decoder_tokens), dtype='float32')
print(decoder_input_data.shape)   # (5892, 298, 1010)

decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_len, num_decoder_tokens), dtype='float32')
print(decoder_target_data.shape)  # (5892, 298, 1010)

# 0으로 채워진 배열에 해당 글자가 있는 지점에는 1을 기억
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
  # print(i, input_text, target_text)  # 9284 Please tidy up your bedroom. 	네 침실 좀 정리해라.
  for t, char in enumerate(input_text): # t:인덱스값, char:characters값
    encoder_input_data[i, t, input_token_index[char]] = 1. # 해당 글자가 있는 경우 1을 기억
  for t, char in enumerate(target_text): # t:인덱스값, char:characters값
    decoder_input_data[i, t, target_token_index[char]] = 1. # 해당 글자가 있는 경우 1을 기억
    if t > 0:
      decoder_target_data[i, t - 1, target_token_index[char]] = 1.  # decoder_target_data가 한 step 앞서 진행됨. 시작 문자는 포함되지 않음
  # print(encoder_input_data[[1]])

input_texts :  ['Go.', 'Hi.', 'Run!', 'Run.', 'Who?', 'Wow!', 'Duck!', 'Fire!', 'Help!', 'Hide.', 'Jump!', 'Jump.', 'Stay.', 'Wait!', 'Wait!', 'Wait.', 'Begin.', 'Hello!', 'Hello.', 'I see.', 'I try.', 'I won!', 'Oh no!', 'Relax.', 'Shoot!', 'Smile.', 'Sorry?', 'Sorry?', 'Sorry?', 'Attack!', 'Attack!', 'Freeze!', 'Get up.', 'Got it!', 'Got it?', 'He ran.', 'Hug me.', 'I know.', 'I lost.', 'I quit.', 'I work.', 'Listen.', 'No way!', 'No way!', 'Thanks.', 'We try.', 'We won.', 'Why me?', 'Awesome!', 'Be calm.', 'Be calm.', 'Be fair.', 'Beat it.', 'Call us.', 'Come in.', 'Come on!', 'Get out.', 'Go away!', 'Go away.', 'Goodbye!', 'Goodbye.', 'He came.', 'He came.', 'He runs.', 'Help me!', 'Help me.', 'Hit Tom.', 'Hold it!', 'Hold it!', 'Hold it!', 'Hold it!', 'I agree.', "I'm sad.", 'Join us.', 'Kiss me.', 'Kiss me.', 'Me, too.', 'Open up.', 'Perfect!', 'Show me.', 'Shut up!', 'Skip it.', 'Stop it.', 'Tell me.', 'Text me.', 'Tom won.', 'Wake up!', 'Wash up.', 'We know.', 'We lost.', 'Welc

In [ ]:
# 입력 시퀀스를 정의 후 처리 : Functional api 사용
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state = True) # return_sequences = True/False에 상관없이 마지막 은닉상태를 출력함
encoder_outputs, state_h, state_c = encoder(encoder_inputs)  # state_h:은닉 상태, state_c:셀 상태

# encoder_outputs은 버리고 state_h, state_c만 유지 : context vector
encoder_states = [state_h, state_c]  # 이게 바로 context vector(문맥 벡터)!!

# 디코더 설계
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)  # 모든 시점의 은닉상태와 마지막 시점의 은닉상태를 출력
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)
# 인코더_입력_데이터 및 디코더_입력_데이터를 디코더_타겟_데이터로 변환하는 모델을 정의!!
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
print(model.summary())

model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size=batch_size,
          epochs=epochs, validation_split=0.2, verbose=2)

model.save('tf41.hdf5')


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, None, 65)]           0         []                            
                                                                                                  
 input_4 (InputLayer)        [(None, None, 713)]          0         []                            
                                                                                                  
 lstm_2 (LSTM)               [(None, 1024),               4464640   ['input_3[0][0]']             
                              (None, 1024),                                                       
                              (None, 1024)]                                                       
                                                                                            

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# 번역 동작 단계
# 1) 번역하고자 하는 입력문장이 인코더에 들어와서 은닉상태와 셀 상태를 얻는다.
# 2) 상태와 <sos>에 해당하는 '\t'를 디코더로 보낸다.
# 3) 디코더가 <eos>에 해당하는 '\n'이 나올 때까지  다음 문자를 예측하는 행동을 반복한다.

# seq2seq 번역기 동작시키기
encoder_model = Model(encoder_inputs, encoder_states) # 인코더 정의

# 디코더 설계
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_state_inputs = [decoder_state_input_h, decoder_state_input_c ]
# 문장의 다음 단어 예측하기 위해 초기상태를 이전 시점의 상태로 사용한다.
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_state_inputs)
decoder_states = [state_h, state_c]  # 훈련상태와 달리 LSTM이 반환하는 은닉상태와 셀 상태를 가진 state_h, state_c를 기억
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_state_inputs, [decoder_outputs] + decoder_states)

# 토큰 인덱스를 역방향으로 조회하여 시퀀스를 디코딩하기. 글자로부터 인덱스를 얻는 것이 아니라 인덱스에서 글자 얻기
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())
print(reverse_input_char_index)
print(reverse_target_char_index)

{0: ' ', 1: '!', 2: "'", 3: ',', 4: '-', 5: '.', 6: '0', 7: '1', 8: '2', 9: '3', 10: '4', 11: '5', 12: '7', 13: ':', 14: '?', 15: 'A', 16: 'B', 17: 'C', 18: 'D', 19: 'E', 20: 'F', 21: 'G', 22: 'H', 23: 'I', 24: 'J', 25: 'K', 26: 'L', 27: 'M', 28: 'N', 29: 'O', 30: 'P', 31: 'Q', 32: 'R', 33: 'S', 34: 'T', 35: 'U', 36: 'V', 37: 'W', 38: 'Y', 39: 'a', 40: 'b', 41: 'c', 42: 'd', 43: 'e', 44: 'f', 45: 'g', 46: 'h', 47: 'i', 48: 'j', 49: 'k', 50: 'l', 51: 'm', 52: 'n', 53: 'o', 54: 'p', 55: 'q', 56: 'r', 57: 's', 58: 't', 59: 'u', 60: 'v', 61: 'w', 62: 'x', 63: 'y', 64: 'z'}
{0: '\t', 1: '\n', 2: ' ', 3: '!', 4: ',', 5: '.', 6: '0', 7: '1', 8: '2', 9: '3', 10: '4', 11: '5', 12: '7', 13: ':', 14: '?', 15: '~', 16: '가', 17: '각', 18: '간', 19: '갇', 20: '갈', 21: '감', 22: '갑', 23: '값', 24: '갔', 25: '강', 26: '갖', 27: '같', 28: '갛', 29: '개', 30: '갰', 31: '걔', 32: '거', 33: '걱', 34: '건', 35: '걷', 36: '걸', 37: '검', 38: '겁', 39: '것', 40: '게', 41: '겐', 42: '겠', 43: '겨', 44: '격', 45: '견', 46: '결', 47: '겼',

In [8]:
def decode_sequence(input_seq):
  # 입력으로부터 인코더의 상태를 얻음
  states_value = encoder_model.predict(input_seq, verbose=0)

  # <SOS>에 해당하는 원-핫 벡터 생성
  target_seq = np.zeros((1, 1, num_decoder_tokens))
  target_seq[0, 0, target_token_index['\t']] = 1.

  stop_condition = False
  decoded_sentence = ""

  # stop_condition이 True가 될 때까지 루프 반복
  while not stop_condition:
    # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
    output_tokens, h, c = decoder_model.predict([target_seq] + states_value, verbose=0)

    # 예측 결과를 문자로 변환
    sampled_token_index = np.argmax(output_tokens[0, -1, :])
    sampled_char = reverse_target_char_index[sampled_token_index]

    # 현재 시점의 예측 문자를 예측 문장에 추가
    decoded_sentence += sampled_char

    # <eos>에 도달하거나 최대 길이를 넘으면 중단.
    if (sampled_char == '\n' or
        len(decoded_sentence) > max_decoder_seq_len):
        stop_condition = True

    # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, sampled_token_index] = 1.

    # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
    states_value = [h, c]

  return decoded_sentence
for seq_index in [0,3,5,10,50]: # 입력 문장의 인덱스
  input_seq = encoder_input_data[seq_index:seq_index+1]
  decoded_sentence = decode_sequence(input_seq)
  print(35 * "-")
  print('입력 문장:', input_texts[seq_index])
  print('번역 문장:', decoded_sentence) # '\n'을 빼고 출력

-----------------------------------
입력 문장: Go.
번역 문장: 날 !.

-----------------------------------
입력 문장: Run.
번역 문장: 저어.

-----------------------------------
입력 문장: Wow!
번역 문장: 누어.

-----------------------------------
입력 문장: Jump!
번역 문장: 점어!

-----------------------------------
입력 문장: Be calm.
번역 문장: 침착!

